StrOutputParser

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")
 
chat_model = ChatOpenAI(model="qwen-plus")

response = chat_model.invoke("什么是大语言模型?")
print(type(response))

# print(response.content)

parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))
print(str_response)

JsonOutputParser

In [14]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template = ChatPromptTemplate.from_messages([
    ('system','你是一个靠谱的{role}'),
    ('human',"{question}")
])

prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说?问题用q表示，答案用a表示，返回一个JSON格式的数据"})

# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说?"})

response = chat_model.invoke(prompt)
print(response.content)

parser = JsonOutputParser()
json_response = parser.invoke(response)
print(json_response)

chain = chat_prompt_template | chat_model | parser
json_result1 = chain.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说?问题用q表示，答案用a表示，返回一个JSON格式的数据"})
print(json_result1)

{
  "q": "人工智能用英文怎么说?",
  "a": "Artificial Intelligence"
}
{'q': '人工智能用英文怎么说?', 'a': 'Artificial Intelligence'}
{'q': '人工智能用英文怎么说?', 'a': 'Artificial Intelligence'}


In [15]:
from langchain_core.prompts import PromptTemplate

joke_query = "告诉我一个笑话."

parser = JsonOutputParser()

prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions":parser.get_format_instructions()},
)

prompt = prompt_template.invoke(input={"question":joke_query})
response = chat_model.invoke(prompt)
json_result = parser.invoke(response)
print(json_result)

chain = prompt_template | chat_model | parser
json_result1 = chain.invoke(input={"question":joke_query})
print(json_result1)

{'joke': '有一天，小明去面试，面试官问他：‘你有什么特长？’ 小明说：‘我会预测未来。’ 面试官笑了笑：‘那你预测一下，你什么时候能被录用？’ 小明淡定地说：‘这个嘛……我预测我不会被录用。’ 面试官一愣：‘你预测错了，你被录用了！’ 小明摇摇头：‘不，我预测对了——我根本不想来上班。’'}
{'joke': '有一天，小明去面试，老板问：‘你有什么特长？’ 小明回答：‘我会预测未来。’ 老板笑了笑：‘那你预测一下，你什么时候能被录用？’ 小明淡定地说：‘这个嘛……我预测我不会被录用。’ 老板一愣：‘你预测错了，你被录用了！’ 小明摇摇头：‘不，我预测对了——因为我根本不想来上班。’'}


知识的扩展：chain

In [16]:
response = chat_model.invoke("请生成周星驰的简短电影记录，将影片附在<movie></movie>标签中")

print(response.content)

<movie>
周星驰是一位著名的香港演员、导演和编剧，以其独特的无厘头喜剧风格闻名。他主演并参与创作了多部经典华语电影，代表作包括《大话西游》《功夫》《少林足球》《喜剧之王》《食神》等。他的作品融合夸张表演、讽刺幽默与深刻情感，影响了一代又一代观众，并在国际上获得广泛认可。
</movie>


In [17]:
from langchain_core.output_parsers import XMLOutputParser

parser = XMLOutputParser()
print(parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


In [4]:
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate

actor_query = "生成汤姆·汉克斯的简短电影记录,使用中文回复"

parser = XMLOutputParser()

prompt_template1 = PromptTemplate.from_template(
    template="用户的问题:{query}\n使用的格式:{format_instructions}"
)

prompt_template2 = prompt_template1.partial(format_instructions=parser.get_format_instructions())

response = chat_model.invoke(prompt_template2.invoke(input={"query":actor_query}))
# print(response.content)

xml_result = parser.invoke(response)
print(xml_result)

{'movie_record': [{'actor': '汤姆·汉克斯'}, {'known_for': '美国著名演员、制片人，两届奥斯卡最佳男主角奖得主'}, {'notable_films': [{'film': '阿甘正传'}, {'film': '西雅图夜未眠'}, {'film': '拯救大兵瑞恩'}, {'film': '费城故事'}, {'film': '荒岛余生'}]}, {'awards': [{'oscar': '2次奥斯卡最佳男主角奖（《费城故事》《阿甘正传》）'}, {'golden_globe': '多次金球奖'}]}, {'career_span': '1979年至今'}]}


列表解析器CommaSeperatedListOutputParser

In [7]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

messages = "大象,猩猩,狮子"
result = output_parser.parse(messages)
print(result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['大象', '猩猩', '狮子']
<class 'list'>


In [8]:
output_parser = CommaSeparatedListOutputParser()

chat_prompt = PromptTemplate.from_template(
    "生成5个关于{text}的列表.\n\n{format_instructions}",
    partial_variables={
        "format_instructions":output_parser.get_format_instructions()
    }
)

chain = chat_prompt | chat_model | output_parser
res = chain.invoke({"text":"电影"})
print(res)
print(type(res))

['经典电影推荐', '科幻电影精选', '恐怖电影排行榜', '动画电影佳作', '奥斯卡获奖影片']
<class 'list'>


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser

chat_prompt = ChatPromptTemplate.from_messages([
    ("system","{format_instructions}"),
    ("human","{request}")
])

output_parser = DatetimeOutputParser()

chain = chat_prompt | chat_model | output_parser
res = chain.invoke({
    "request":"中华人民共和国是什么时候成立的",
    "format_instructions":output_parser.get_format_instructions()
})

print(res)
print(type(res))


1949-10-01 00:00:00
<class 'datetime.datetime'>
